In [114]:
"""
Members : 

- EKANE Emile
- LOUOKDOM FOKAM Neal
- SIEWE DAHE William
- NZEKET Aïcha

"""

'\nMembers : \n\n- Ekane Emile\n- LOUOKDOM FOKAM Neal\n- SIEWE DAHE William\n- NZEKET Aïcha\n\n'

In [115]:
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import glob
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import random 
pyLDAvis.enable_notebook()
%matplotlib inline

In [116]:
df = pd.read_csv('question_responce.csv')
df = df[["responce", "question"]]

In [117]:
def cleaningfunc (column):
    cleaned_data = []
    sentence = []
    data = []

    for line in column:
        cleaned_data.append(re.sub(r'[^\w\s]', ' ',re.sub(r"(?:\@|https?\://)\S+", "", re.sub("#[A-Za-z0-9_]+","", line)).lower()))

    for line in cleaned_data:
        for word in line.split():
            if word not in stopwords.words("english"):
                sentence.append(word)
        data.append((" ").join(sentence))
        sentence = []
    cleaned_data = data
    
    return cleaned_data

In [118]:
def tokenizingfunc(data_cleaned):
    data_tokenized = [word_tokenize(line) for line in data_cleaned]
    data_tokenized = [word for word in data_tokenized if word not in stopwords.words("english")]
    return data_tokenized

def tokenizingfuncquestion(text):
    cleaned_data = []
    data = re.sub(r'[^\w\s]', ' ',re.sub(r"(?:\@|https?\://)\S+", "", re.sub("#[A-Za-z0-9_]+","", text)).lower())
    
    for word in data.split():
        if word not in stopwords.words("english"):
                cleaned_data.append(word)
    
    return cleaned_data

In [119]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return ''

def lemmatizingfunc(data_tokenized):
    lemmatizer = WordNetLemmatizer()

    data_lemmatized = []
    line_lemmatized = []

    for line in data_tokenized:
        for i, word in enumerate(line):
            pos = get_wordnet_pos(pos_tag([word])[0][1])
            if pos != '':
                line_lemmatized.append(lemmatizer.lemmatize(word, pos))
            else:
                line_lemmatized.append(word)
        data_lemmatized.append(line_lemmatized)
        line_lemmatized = []
    
    return data_lemmatized

def lemmatizingfuncquestion(data_tokenized):
    lemmatizer = WordNetLemmatizer()

    data_lemmatized = []

    for word in data_tokenized:
        pos = get_wordnet_pos(pos_tag([word])[0][1])
        if pos != '':
            data_lemmatized.append(lemmatizer.lemmatize(word, pos))
        else:
            data_lemmatized.append(word)
    
    return data_lemmatized
            

In [120]:
def preprocessingquestion(text):
    return lemmatizingfuncquestion(tokenizingfuncquestion(text))

In [121]:
question_cleaned = cleaningfunc(df["question"])
question_tokenized = tokenizingfunc(question_cleaned)
question_lemmatized = lemmatizingfunc(question_tokenized)

response_cleaned = cleaningfunc(df["responce"])
response_tokenized = tokenizingfunc(response_cleaned)
response_lemmatized = lemmatizingfunc(response_tokenized)

In [122]:
def doc2vecfunc():
    documents = glob.glob("corpus/*")
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['responce'])]
    model = Doc2Vec(vector_size=10, alpha=0.025,min_alpha=0.00025,min_count=10,dm=0, epochs=150)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [123]:
def ldafunc(data_lemmatized):
    id2word = corpora.Dictionary(data_lemmatized)
    
    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    lda = LdaMulticore(corpus = corpus, id2word=id2word, num_topics=25, passes=100)
    
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda, corpus, id2word)

    return lda,corpus,vis

In [124]:
model = doc2vecfunc()
lda,corpus,vis = ldafunc(question_lemmatized)

/usr/local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [125]:
print('Perplexity: ', lda.log_perplexity(corpus)) 

Perplexity:  -8.134501835893126


In [126]:
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"dm" + 0.023*"sent" + 0.014*"get" + 0.014*"back" + 0.014*"flight" + 0.013*"would" + 0.011*"ok" + 0.011*"issue" + 0.009*"late" + 0.008*"like"
Topic: 1 
Words: 0.019*"flight" + 0.014*"take" + 0.014*"seat" + 0.012*"1" + 0.009*"time" + 0.009*"people" + 0.008*"one" + 0.007*"make" + 0.007*"plane" + 0.007*"thank"
Topic: 2 
Words: 0.035*"flight" + 0.015*"get" + 0.011*"seat" + 0.010*"know" + 0.009*"tomorrow" + 0.009*"amp" + 0.009*"early" + 0.008*"update" + 0.008*"customer" + 0.008*"delay"
Topic: 3 
Words: 0.021*"seat" + 0.016*"flight" + 0.010*"get" + 0.010*"card" + 0.009*"thanks" + 0.009*"exit" + 0.008*"row" + 0.008*"gate" + 0.006*"way" + 0.006*"home"
Topic: 4 
Words: 0.020*"day" + 0.018*"one" + 0.016*"ticket" + 0.012*"way" + 0.011*"get" + 0.010*"fly" + 0.009*"thank" + 0.009*"try" + 0.009*"airline" + 0.009*"need"
Topic: 5 
Words: 0.043*"flight" + 0.036*"hour" + 0.027*"miss" + 0.018*"make" + 0.015*"2" + 0.015*"bad" + 0.014*"delayed" + 0.014*"delay" + 0.013*"3" + 0.011*"get

In [127]:
ourData = {"intents":[
    {
        "tag" : "Thanks",
        "patterns" : ["thanks", "thanksgiving", "thank", "happy", "great", "beautiful", "nice"],
        "responses" : ["I am happy to hear that", "We will continue to work hard to maintain your satifaction"]
    },
    {
        "tag" : "Delay",
        "patterns" : ["delay", "late", "wait"],
        "responses" : ["We are really sorry to hear that", "We sincerely apologize about this delay"]
    },
    {
        "tag" : "Cancel",
        "patterns" : ["cancel"],
        "responses" : ["We're sorry about that inconvenient situation. Rebook another flight"]
    },
    {
        "tag" : "Guidance",
        "patterns" : ["guide"],
        "responses" : ["We'll see what we could do", "We'll send you someone as soon as possible"]
    },
    {
        "tag" : "Goodbye",
        "patterns" : ["bye", "goodbye"],
        "responses" : ["You're welcome, goodbye and have a nice stay"]
    },
    {
        "tag" : "Please",
        "patterns" : ["please"],
        "responses" : ["I will do the best I can"]
    },
    {
        "tag" : "Disappointment",
        "patterns" : ["disappoint", "upset"],
        "responses" : ["We are really sorry to hear that, we shall do better next time"]
    },
    {
        "tag" : "Refund",
        "patterns" : ["refund"],
        "responses" : ["We are really sorry, our team will study your request and then, give you a feedback"]
    },
    {
        "tag" : "Help",
        "patterns" : ["help"],
        "responses" : ["I'll do my best for that", "We'll send you someone asap"]
    }
]}

In [128]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.078762 -0.000765       1        1  7.117201
14    -0.119236  0.008669       2        1  5.847649
10    -0.070846 -0.075283       3        1  5.752671
17    -0.065767  0.148093       4        1  5.301824
21    -0.144867 -0.107547       5        1  5.125017
2     -0.061570  0.009553       6        1  4.650724
23     0.022323 -0.048093       7        1  4.561282
6     -0.033817  0.008156       8        1  4.520717
9     -0.011220  0.101734       9        1  4.505033
15    -0.020727  0.036135      10        1  4.316659
5     -0.099056 -0.089131      11        1  4.180278
4      0.066092  0.017630      12        1  3.943614
20     0.062675  0.022493      13        1  3.782138
11     0.078953 -0.074152      14        1  3.638135
3      0.031419 -0.027712      15        1  3.513558
7      0.042497 -0.015922      16        1  3.408161
1      0.004981  0.040265      17        1  3.360356
12    -0.024039  0.116131      18        1  3.308520
8      0.051426  0.014892      19        1  3.198876
19     0.031078  0.024881      20        1  3.055204
18     0.029983  0.053444      21        1  2.836597
24     0.067121 -0.061686      22        1  2.754083
13     0.017835 -0.014726      23        1  2.720502
0      0.065905 -0.078509      24        1  2.543440
22     0.157621 -0.008550      25        1  2.057759, topic_info=        Term        Freq       Total Category  logprob  loglift
205   cancel   66.000000   66.000000  Default  30.0000  30.0000
254      bag  114.000000  114.000000  Default  29.0000  29.0000
54      hour  104.000000  104.000000  Default  28.0000  28.0000
29    flight  548.000000  548.000000  Default  27.0000  27.0000
206  delayed   55.000000   55.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
255   people    1.598351   32.695239  Topic25  -5.5126   0.8653
419     work    1.598351   43.534203  Topic25  -5.5126   0.5790
415     like    1.598351   51.640111  Topic25  -5.5126   0.4082
13   airline    1.598351   67.779993  Topic25  -5.5126   0.1363
457      hey    1.598351   37.097765  Topic25  -5.5126   0.7390

[1891 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
441       3  0.295217              0
441       7  0.590433              0
2281     22  0.608108  0012147175635
2282     22  0.608108  0012158376700
2148     10  0.591472             01
...     ...       ...            ...
1581      3  0.589047          youre
2940     24  0.623402          youth
1731      6  0.589911             yr
1731      8  0.294956             yr
1709     20  0.607193            yvr

[5115 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 15, 11, 18, 22, 3, 24, 7, 10, 16, 6, 5, 21, 12, 4, 8, 2, 13, 9, 20, 19, 25, 14, 1, 23])

In [129]:
for index, score in sorted(lda[corpus[65]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda.print_topic(index, 10)))


Score: 0.9039932489395142	 
Topic: 0.037*"bag" + 0.027*"check" + 0.027*"carry" + 0.019*"overhead" + 0.017*"thanks" + 0.016*"charge" + 0.015*"get" + 0.015*"try" + 0.014*"bin" + 0.014*"seat"


In [130]:
topics = []
for i in range(len(df["responce"])):
    topics.append(sorted(lda[corpus[i]], key = lambda x : -1*x[1])[0][0])

df['topics'] = topics

In [131]:
id = 100
question = lemmatizingfuncquestion(tokenizingfuncquestion(df["question"].loc[id]))

In [132]:
id2word = corpora.Dictionary([question])
corpus = [id2word.doc2bow(question)]
print(lda[corpus])

In [133]:
model.dv.most_similar(model.infer_vector(question))

[(117, 0.893161952495575),
 (975, 0.8799808025360107),
 (1380, 0.8645240664482117),
 (736, 0.8626986742019653),
 (1436, 0.8451420664787292),
 (1313, 0.8388538956642151),
 (1814, 0.828161895275116),
 (1789, 0.8265056610107422),
 (829, 0.8218382000923157),
 (630, 0.8138636946678162)]

In [134]:
print("question : " + df["question"].loc[id] + "\n")
print("response : " + df["responce"].loc[id])

question : @AmericanAir Great thank you. I had a GREAT flight.

response : @119479 We love hearing this and look forward to welcoming you on board again soon.


In [135]:
df["responce"].loc[model.dv.most_similar(model.infer_vector(question))[0][0]]

"@120197 Thanks for the info David. We'll pass this on to our team so she gets the recognition she deserves."

In [139]:
def getRes(text, fJson):
  listOfIntents = fJson["intents"]
  for w in text:
    for i in listOfIntents: 
      for el in i["patterns"]:
        if el == w:
          ourResult = random.choice(i["responses"])
          return ourResult

getRes(question, ourData)

We will continue to work hard to maintain your satifaction


'We will continue to work hard to maintain your satifaction'